In [27]:
import glob
import torch
from PIL import Image
import torch.nn as nn
import torch
from torchvision import transforms
import torch.nn.functional as F
from torch.optim import Adam
import torchvision.datasets as datasets


In [50]:
files = glob.glob("/home/kamlesh/detasets/caltech101/101_ObjectCategories/*/*")
classes = glob.glob("/home/kamlesh/detasets/caltech101/101_ObjectCategories/*")
classes = [classname.split('/')[-1] for classname in classes ]
print(len(classes))#102 including background class
print(len(files))

102
9145


In [51]:
img = Image.open(files[0])
print(files[0])
print(img.size)

/home/kamlesh/detasets/caltech101/101_ObjectCategories/mandolin/image_0029.jpg
(300, 158)


In [15]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [63]:
X = []
y = []
for file in files[:100]:
    img = Image.open(file)
    img = img.convert("RGB")
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t,0)
    X.append(batch_t)
    y.append(file.split('/')[-2])

KeyboardInterrupt: 

In [ ]:
# print(X)
print(len(y))

In [28]:
train_set = datasets.ImageFolder("/home/kamlesh/detasets/caltech101/101_ObjectCategories", transform = transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)

In [29]:
print(train_set)

Dataset ImageFolder
    Number of datapoints: 9144
    Root location: /home/kamlesh/detasets/caltech101/101_ObjectCategories
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [30]:
class VGG(nn.Module):
    def __init__(self,n_classes):
        super(VGG,self).__init__()
                                #(in_channel_size,out_channel_Size,kernel_size,padding)
        self.conv1_1 = nn.Conv2d(3,64,kernel_size=3,padding=1)
        self.conv1_2 = nn.Conv2d(64,64,kernel_size=3,padding=1)
        
        self.conv2_1 = nn.Conv2d(64,128,kernel_size=3,padding=1)
        self.conv2_2 = nn.Conv2d(128,128,kernel_size=3,padding=1)
        
        self.conv3_1 = nn.Conv2d(128,256,kernel_size=3,padding=1)
        self.conv3_2 = nn.Conv2d(256,256,kernel_size=3,padding=1)
        self.conv3_3 = nn.Conv2d(256,256,kernel_size=3,padding=1)
        
        self.conv4_1 = nn.Conv2d(256,512,kernel_size=3,padding=1)
        self.conv4_2 = nn.Conv2d(512,512,kernel_size=3,padding=1)
        self.conv4_3 = nn.Conv2d(512,512,kernel_size=3,padding=1)
        
        self.conv5_1 = nn.Conv2d(512,512,kernel_size=3,padding=1)
        self.conv5_2 = nn.Conv2d(512,512,kernel_size=3,padding=1)
        self.conv5_3 = nn.Conv2d(512,512,kernel_size=3,padding=1)
                                    #(kerel_size,stride)
        self.maxpool = nn.MaxPool2d(2,2)
        
        self.fc6 = nn.Linear(7*7*512,4096)
        self.fc7 = nn.Linear(4096,4096)
        self.fc8 = nn.Linear(4096,112)
        
        def forward(self,x,training=True):
            x = F.relu(self.conv1_1(x))
            x = F.relu(Self.conv1_2(x))
            x = self.maxpool(x)
            x = F.relu(self.conv2_1(x))
            x = F.relu(self.conv2_2(x))
            x = self.maxpool(X)
            x = F.relu(self.conv3_1(x))
            x = F.relu(self.conv3_2(x))
            x = F.relu(self.conv3_3(x))
            x = self.maxpool(X)
            x = F.relu(self.conv4_1(x))
            x = F.relu(self.conv4_2(x))
            x = F.relu(self.conv4_3(x))
            x = self.maxpool(x)
            x = F.relu(self.conv5_1(x))
            x = F.relu(self.conv5_2(x))
            x = F.relu(self.conv5_3(x))
            x = self.maxpool(x)
            x = x.view(-1,7*7*512)
            x = F.relu(self.fc6(x))
            x = F.dropout(x,0.5,training=training)
            x = F.relu(self.fc7(x))
            x = F.dropout(x,0.5,training=training)
            x = F.relu(self.fc8(x))
            return x
        def predict(self,x):
            x = F.softmax(self.forward(x,training=False))
            return x
        def accuracy(self,x,y):
            prediction = self.predict(x)
            maxv, index = torch.max(prediction,1)
            acc = 100 * torch.sum(torch.eq(indices.float(), y.float()).float())/y.size()[0]
            return acc.cpu().data[0]
vgg16 = VGG(112)

            

In [31]:
optimizer = Adam(vgg16.parameters(), lr=0.001, weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()
def adjust_learning_rate(epoch):
    lr = 0.001

    if epoch > 180:
        lr = lr / 1000000
    elif epoch > 150:
        lr = lr / 100000
    elif epoch > 120:
        lr = lr / 10000
    elif epoch > 90:
        lr = lr / 1000
    elif epoch > 60:
        lr = lr / 100
    elif epoch > 30:
        lr = lr / 10

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr
        
        
def save_models(epoch):
    torch.save(vgg16.state_dict(), "vgg16model_{}.model".format(epoch))
    print("Chekcpoint saved")
    
    
def test():
    vgg16.eval()
    test_acc = 0.0
    for i, (images, labels) in enumerate(test_loader):

        if cuda_avail:
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())

        # Predict classes using images from the test set
        outputs = vgg16(images)
        _, prediction = torch.max(outputs.data, 1)
        
        test_acc += torch.sum(prediction == labels.data)

    # Compute the average acc and loss over all 10000 test images
    test_acc = test_acc / 10000

    return test_acc

In [38]:

def train(num_epochs):
    best_acc = 0.0

    for epoch in range(num_epochs):
        vgg16.train()
        train_acc = 0.0
        train_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            # Move images and labels to gpu if available

            # Clear all accumulated gradients
            optimizer.zero_grad()
            # Predict classes using images from the test set
            outputs = vgg16(images)
            # Compute the loss based on the predictions and actual labels
            loss = loss_fn(outputs, y_train)
            # Backpropagate the loss
            loss.backward()

            # Adjust parameters according to the computed gradients
            optimizer.step()

            train_loss += loss.cpu().data[0] * images.size(0)
            _, prediction = torch.max(outputs.data, 1)
            
            train_acc += torch.sum(prediction == y_train.data)

        # Call the learning rate adjustment function
        adjust_learning_rate(epoch)

        # Compute the average acc and loss over all 50000 training images
        train_acc = train_acc / 50000
        train_loss = train_loss / 50000

        # Evaluate on the test set
        test_acc = test()

        # Save the model if the test acc is greater than our current best
        if test_acc > best_acc:
            save_models(epoch)
            best_acc = test_acc

        # Print the metrics
        print("Epoch {}, Train Accuracy: {} , TrainLoss: {} , Test Accuracy: {}".format(epoch, train_acc, train_loss,test_acc))
                        

In [39]:
train(10)

torch.Size([32, 3, 224, 224])
tensor([ 57,   4, 101,   4,  63,  87,  95,   2,   4,  98,   2,   4,  94,  84,
          2,  43,  95,  61,  48,   2,   1,  87,   1,   1,  31,  33,  34,  59,
          6,   7,   0,  15])


NotImplementedError: 